In [21]:
import duckdb
import pandas as pd
import warnings
import altair as alt

class AltairDeprecationWarning(Warning):
    pass

warnings.filterwarnings("ignore", category=AltairDeprecationWarning)

# Load your CSV files into pandas DataFrames
population_df = pd.read_csv('data/world_population.csv')
olympic_medals_df = pd.read_csv('data/Summer_olympic_Medals.xls')
gdp_df = pd.read_csv('data/gdp_data.csv')

# Create DuckDB connection
con = duckdb.connect(database=':memory:')

# Load DataFrames into DuckDB
con.execute("CREATE TABLE population AS SELECT * FROM population_df")
con.execute("CREATE TABLE olympic_medals AS SELECT * FROM olympic_medals_df")
con.execute("CREATE TABLE gdp AS SELECT * FROM gdp_df")

# Verify table creation
print(con.execute("SHOW TABLES").df())


             name
0             gdp
1  olympic_medals
2      population


In [22]:
# Describe the tables to check schemas
print(con.execute("DESCRIBE TABLE population").df())
print(con.execute("DESCRIBE TABLE olympic_medals").df())
print(con.execute("DESCRIBE TABLE gdp").df())


                    column_name column_type null   key default extra
0                          Rank      BIGINT  YES  None    None  None
1                          CCA3     VARCHAR  YES  None    None  None
2             Country/Territory     VARCHAR  YES  None    None  None
3                       Capital     VARCHAR  YES  None    None  None
4                     Continent     VARCHAR  YES  None    None  None
5               2022 Population      BIGINT  YES  None    None  None
6               2020 Population      BIGINT  YES  None    None  None
7               2015 Population      BIGINT  YES  None    None  None
8               2010 Population      BIGINT  YES  None    None  None
9               2000 Population      BIGINT  YES  None    None  None
10              1990 Population      BIGINT  YES  None    None  None
11              1980 Population      BIGINT  YES  None    None  None
12              1970 Population      BIGINT  YES  None    None  None
13                   Area (km²)   

In [23]:
import altair as alt

# Query for the top 10 countries by medal count
medal_counts = con.execute("""
    SELECT Country_Name, SUM(Gold) AS Gold, SUM(Silver) AS Silver, SUM(Bronze) AS Bronze
    FROM olympic_medals
    GROUP BY Country_Name
    ORDER BY Gold DESC
    LIMIT 10
""").df()

# Melt the DataFrame for easier plotting
medal_counts_melted = medal_counts.melt(id_vars='Country_Name', var_name='Medal', value_name='Count')

# Create the bar chart
bar_chart = alt.Chart(medal_counts_melted).mark_bar().encode(
    x=alt.X('Country_Name:N', sort='-y'),
    y='Count:Q',
    color='Medal:N',
    tooltip=['Country_Name', 'Medal', 'Count']
).properties(
    title='Top 10 Countries by Medal Count'
)

bar_chart.show()


alt.Chart(...)

In [24]:
# Specify the country
country_name = "United States"

# Query for GDP, population, and medals over the years
country_data = con.execute(f"""
    SELECT gdp.Year, gdp.GDP, gdp."GDP-Per-Capita", gdp."GDP-Growth", pop."2022 Population", 
           (medals.Gold + medals.Silver + medals.Bronze) AS Total_Medals
    FROM gdp
    JOIN population pop ON gdp.Country = pop."Country/Territory"
    JOIN olympic_medals medals ON gdp.Country = medals.Country_Name AND gdp.Year = medals.Year
    WHERE gdp.Country = '{country_name}'
    ORDER BY gdp.Year
""").df()

# Create the line chart
line_chart = alt.Chart(country_data).mark_line().encode(
    x='Year:O',
    y='Total_Medals:Q',
    tooltip=['Year', 'Total_Medals', 'GDP', '2022 Population']
).properties(
    title=f'Medal Count for {country_name} Over the Years'
)

line_chart.show()


alt.Chart(...)

In [25]:
# Check the distinct years available in the olympic_medals table
distinct_years = con.execute("SELECT DISTINCT Year FROM olympic_medals").df()
print(distinct_years)


    Year
0   1896
1   1912
2   1928
3   1948
4   1988
5   1996
6   1924
7   1956
8   1984
9   1960
10  1992
11  2000
12  1904
13  1976
14  1980
15  2008
16  1900
17  1920
18  1936
19  1952
20  1968
21  2016
22  2004
23  2020
24  1908
25  1932
26  1964
27  1972
28  2012


In [26]:
# Query to get the total medals for each country per year
medal_data = con.execute("""
    SELECT Year, Country_Name, SUM(Gold) AS Gold, SUM(Silver) AS Silver, SUM(Bronze) AS Bronze, 
           (SUM(Gold) + SUM(Silver) + SUM(Bronze)) AS Total_Medals
    FROM olympic_medals
    GROUP BY Year, Country_Name
    ORDER BY Year
""").df()

# Print the data to check its contents
print(medal_data.head())
print(medal_data['Year'].unique())


   Year   Country_Name  Gold  Silver  Bronze  Total_Medals
0  1896  United States  11.0     7.0     2.0          20.0
1  1896        Germany   6.0     5.0     2.0          13.0
2  1896     Mixed team   1.0     0.0     1.0           2.0
3  1896      Australia   2.0     0.0     0.0           2.0
4  1896        Denmark   1.0     2.0     3.0           6.0
[1896 1900 1904 1908 1912 1920 1924 1928 1932 1936 1948 1952 1956 1960
 1964 1968 1972 1976 1980 1984 1988 1992 1996 2000 2004 2008 2012 2016
 2020]


In [27]:
# Check a sample of the data from the olympic_medals table
print(con.execute("SELECT * FROM olympic_medals LIMIT 10").df())


   Year Host_country Host_city   Country_Name Country_Code  Gold  Silver  \
0  1896       Greece    Athens  Great Britain          GBR     2       3   
1  1896       Greece    Athens        Hungary          HUN     2       1   
2  1896       Greece    Athens         France          FRA     5       4   
3  1896       Greece    Athens  United States          USA    11       7   
4  1896       Greece    Athens        Germany          GER     6       5   
5  1896       Greece    Athens        Austria          AUT     2       1   
6  1896       Greece    Athens     Mixed team          ZZX     1       0   
7  1896       Greece    Athens         Greece          GRE    10      18   
8  1896       Greece    Athens    Switzerland          SUI     1       2   
9  1896       Greece    Athens      Australia          AUS     2       0   

   Bronze  
0       2  
1       3  
2       2  
3       2  
4       2  
5       2  
6       1  
7      19  
8       0  
9       0  


In [28]:

# Query to get the total medals for each country per year
medal_data = con.execute("""
    SELECT Year, Country_Name, SUM(Gold) AS Gold, SUM(Silver) AS Silver, SUM(Bronze) AS Bronze, 
           (SUM(Gold) + SUM(Silver) + SUM(Bronze)) AS Total_Medals
    FROM olympic_medals
    GROUP BY Year, Country_Name
""").df()

# Create a selection dropdown for the year
year_selection = alt.selection_point(
    fields=['Year'],
    bind=alt.binding_select(options=sorted(medal_data['Year'].unique().tolist()), name='Select Year: ')
)

# Create the bar chart
bar_chart = alt.Chart(medal_data).mark_bar().encode(
    x=alt.X('Country_Name:N', title='Country'),
    y=alt.Y('Total_Medals:Q', title='Total Medals'),
    color=alt.Color('Country_Name:N', legend=None),
    tooltip=['Year', 'Country_Name', 'Gold', 'Silver', 'Bronze', 'Total_Medals']
).transform_filter(
    year_selection
).properties(
    title='Total Medals by Country and Year'
).add_params(
    year_selection
)

bar_chart.show()


alt.Chart(...)